## Smooth pursuit

In [1]:
# Require libraries
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tsmoothie.smoother import LowessSmoother
from pyts.image import GramianAngularField
from pyts.image import MarkovTransitionField
import glob
from pathlib import Path



### Image Time Sries(ITS)

#### Create path for HC

In [73]:
# Create path for healthy control(HC)
path = os.getcwd()
dirname = os.path.dirname(path)
option = ['URLStart','URLEnd'] # This is the values in the Studio event column where stimuli start and end events from dataset
directory_hc  = dirname+"/data/subjects/HC"
s_path = dirname+"/result/4_dps/s_hc" # single gasf path for hc
c_path = dirname+"/result/4_dps/c_hc" # combine gasf path for pd
files_hc = glob.iglob(directory_hc+'/[!.]*') # skip unwanted file such as .DS_Store file

In [74]:
# Funtion for combine images
def concat_images(imga, imgb):
    """
    Combines two color image ndarrays side-by-side.
    """
    ha,wa = imga.shape[:2]
    hb,wb = imgb.shape[:2]
    max_height = np.max([ha, hb])
    total_width = wa+wb
    new_img = np.zeros(shape=(max_height, total_width, 3))
    new_img[:ha,:wa]=imga
    new_img[:hb,wa:wa+wb]=imgb
    return new_img

def concat_n_images(image_path_list):
    """
    Combines N color images from a list of image paths.
    """
    output = None
    for i, img_path in enumerate(image_path_list):
        img = plt.imread(img_path)[:,:,:3]
        if i==0:
            output = img
        else:
            output = concat_images(output, img)
    return output

# Function to remove outliers
def outlier_remove(df,eye):
    data = df[eye].interpolate()
    data = data.dropna()
    data = data.values.reshape(1,-1)
    smoother = LowessSmoother(smooth_fraction=0.1, iterations=1)
    smoother.smooth(data)
    low, up = smoother.get_intervals('prediction_interval')
    points = smoother.data[0]
    up_points = up[0]
    low_points = low[0]
    outlier = []
    for i in range(len(points)-1, 0, -1):
        current_point = points[i]
        current_up = up_points[i]
        current_low = low_points[i]
        if current_point > current_up or current_point < current_low:
            outlier.append(current_point)
    return outlier



### Generate images for HC

In [71]:
## Data preporcessing and transforming into images
for file_hc in files_hc:

    # Data preporcessing
    # Interpolate the NaN values with linear
    data_hc  = pd.read_excel(file_hc+'/1_dps.xlsx')#.interpolate()
    t = data_hc[data_hc['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
    data_eye_hc_first = data_hc[t[0]+902:t[0]+27904] # first trial index
    data_eye_hc_second = data_hc[t[2]+902:t[2]+27904] # second trail index

    # First Trial
    outlier_r = outlier_remove(data_eye_hc_first,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_hc_first,'GazePointLeftX (ADCSpx)')

    
    ## Transform times series to images
    # Compute Gramian angular fields, GASF
    # #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_hc_first.drop(data_eye_hc_first.index[data_eye_hc_first['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_first['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'1deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_hc_first.drop(data_eye_hc_first.index[data_eye_hc_first['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_first['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'1deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'1deg'+'right.png',s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'1deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'1deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Second Trial
    outlier_r = outlier_remove(data_eye_hc_second,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_hc_second,'GazePointLeftX (ADCSpx)')

    # Compute Gramian angular fields, GASF
    #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_hc_second.drop(data_eye_hc_second.index[data_eye_hc_second['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_second['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'1deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_hc_second.drop(data_eye_hc_second.index[data_eye_hc_second['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_second['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'1deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #Combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'1deg'+'right.png',s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'1deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'1deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
  

In [75]:
## Data preporcessing and transforming into images
for file_hc in files_hc:

    # Data preporcessing
    # Interpolate the NaN values with linear
    data_hc  = pd.read_excel(file_hc+'/2_dps.xlsx')#.interpolate()
    t = data_hc[data_hc['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
    data_eye_hc_first = data_hc[t[0]+902:t[0]+14404] # first trial index
    data_eye_hc_second = data_hc[t[2]+902:t[2]+14404] # second trail index

    # First Trial
    outlier_r = outlier_remove(data_eye_hc_first,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_hc_first,'GazePointLeftX (ADCSpx)')

    
    ## Transform times series to images
    # Compute Gramian angular fields, GASF
    # #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_hc_first.drop(data_eye_hc_first.index[data_eye_hc_first['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_first['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'2deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_hc_first.drop(data_eye_hc_first.index[data_eye_hc_first['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_first['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'2deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'2deg'+'right.png',s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'2deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'2deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Second Trial
    outlier_r = outlier_remove(data_eye_hc_second,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_hc_second,'GazePointLeftX (ADCSpx)')

    # Compute Gramian angular fields, GASF
    #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_hc_second.drop(data_eye_hc_second.index[data_eye_hc_second['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_second['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'2deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_hc_second.drop(data_eye_hc_second.index[data_eye_hc_second['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_second['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'2deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #Combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'2deg'+'right.png',s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'2deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'2deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
  

In [41]:
## Data preporcessing and transforming into images
for file_hc in files_hc:

    # Data preporcessing
    # Interpolate the NaN values with linear
    data_hc  = pd.read_excel(file_hc+'/4_dps.xlsx')#.interpolate()
    t = data_hc[data_hc['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
    data_eye_hc_first = data_hc[t[0]+902:t[0]+7804] # first trial index
    data_eye_hc_second = data_hc[t[2]+902:t[2]+7804] # second trail index

    # First Trial
    outlier_r = outlier_remove(data_eye_hc_first,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_hc_first,'GazePointLeftX (ADCSpx)')

    
    ## Transform times series to images
    # Compute Gramian angular fields, GASF
    # #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_hc_first.drop(data_eye_hc_first.index[data_eye_hc_first['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_first['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_hc_first.drop(data_eye_hc_first.index[data_eye_hc_first['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_first['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'right.png',s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Second Trial

    outlier_r = outlier_remove(data_eye_hc_second,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_hc_second,'GazePointLeftX (ADCSpx)')

    # Compute Gramian angular fields, GASF
    #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_hc_second.drop(data_eye_hc_second.index[data_eye_hc_second['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_second['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_hc_second.drop(data_eye_hc_second.index[data_eye_hc_second['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_second['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #Combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'right.png',s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
  

## 6 Deg

In [62]:
## Data preporcessing and transforming into images
for file_hc in files_hc:

    # Data preporcessing
    # Interpolate the NaN values with linear
    data_hc  = pd.read_excel(file_hc+'/6_dps.xlsx')#.interpolate()
    t = data_hc[data_hc['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
    data_eye_hc_first = data_hc[t[0]+902:t[0]+5404] # first trial index
    data_eye_hc_second = data_hc[t[2]+902:t[2]+5404] # second trail index

    # First Trial
    outlier_r = outlier_remove(data_eye_hc_first,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_hc_first,'GazePointLeftX (ADCSpx)')

    
    ## Transform times series to images
    # Compute Gramian angular fields, GASF
    # #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_hc_first.drop(data_eye_hc_first.index[data_eye_hc_first['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_first['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'6deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_hc_first.drop(data_eye_hc_first.index[data_eye_hc_first['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_first['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'6deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'6deg'+'right.png',s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'6deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'6deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Second Trial
    outlier_r = outlier_remove(data_eye_hc_second,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_hc_second,'GazePointLeftX (ADCSpx)')

    # Compute Gramian angular fields, GASF
    #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_hc_second.drop(data_eye_hc_second.index[data_eye_hc_second['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_second['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'6deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_hc_second.drop(data_eye_hc_second.index[data_eye_hc_second['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_second['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'6deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #Combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'6deg'+'right.png',s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'6deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'6deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
  

## 8deg

In [67]:
## Data preporcessing and transforming into images
for file_hc in files_hc:

    # Data preporcessing
    # Interpolate the NaN values with linear
    data_hc  = pd.read_excel(file_hc+'/8_dps.xlsx')#.interpolate()
    t = data_hc[data_hc['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
    data_eye_hc_first = data_hc[t[0]+902:t[0]+4204] # first trial index
    data_eye_hc_second = data_hc[t[2]+902:t[2]+4204] # second trail index

    # First Trial
    outlier_r = outlier_remove(data_eye_hc_first,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_hc_first,'GazePointLeftX (ADCSpx)')

    
    ## Transform times series to images
    # Compute Gramian angular fields, GASF
    # #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_hc_first.drop(data_eye_hc_first.index[data_eye_hc_first['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_first['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'8deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_hc_first.drop(data_eye_hc_first.index[data_eye_hc_first['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_first['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'8deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'8deg'+'right.png',s_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'8deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_hc))+'_'+'1'+'8deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Second Trial
    outlier_r = outlier_remove(data_eye_hc_second,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_hc_second,'GazePointLeftX (ADCSpx)')

    # Compute Gramian angular fields, GASF
    #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_hc_second.drop(data_eye_hc_second.index[data_eye_hc_second['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_second['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'8deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_hc_second.drop(data_eye_hc_second.index[data_eye_hc_second['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_hc_second['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'8deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #Combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'8deg'+'right.png',s_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'8deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_hc))+'_'+'2'+'8deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
  

#### Create path for PD

In [76]:
# Create path fo pd
path = os.getcwd()
dirname = os.path.dirname(path)
option = ['URLStart','URLEnd']
directory_pd  = dirname+"/data/subjects/PD"
s_path = dirname+"/result/4_dps/s_pd"
c_path = dirname+"/result/4_dps/c_pd"
option = ['URLStart','URLEnd']
files_pd = glob.iglob(directory_pd+'/[!.]*') # skip unwanted file such as .DS_Store file

#### Generate images for PD

In [ ]:
## Data preporcessing and transforming into images
for file_pd in files_pd:

    # Data preporcessing
    # Interpolate the NaN values with linear
    data_pd  = pd.read_excel(file_pd+'/1_dps.xlsx').interpolate()
    t = data_pd[data_pd['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
    data_eye_pd_first = data_pd[t[0]+902:t[0]+27904] # first trial index
    data_eye_pd_second = data_pd[t[2]+902:t[2]+27904] # second trail index

    # First Trial
    outlier_r = outlier_remove(data_eye_pd_first,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_pd_first,'GazePointLeftX (ADCSpx)')

    
    ## Transform times series to images
    # Compute Gramian angular fields, GASF
    # #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_pd_first.drop(data_eye_pd_first.index[data_eye_pd_first['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_first['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'1deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_pd_first.drop(data_eye_pd_first.index[data_eye_pd_first['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_first['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'1deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'1deg'+'right.png',s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'1deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'1deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Second Trial
    data_eye_pd_second_right = data_eye_pd_second['GazePointRightX (ADCSpx)']
    data_eye_pd_second_left = data_eye_pd_second['GazePointLeftX (ADCSpx)']

    # Compute Gramian angular fields, GASF
    #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_pd_second.drop(data_eye_pd_second.index[data_eye_pd_second['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_second['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'1deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_pd_second.drop(data_eye_pd_second.index[data_eye_pd_second['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_second['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'1deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #Combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'1deg'+'right.png',s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'1deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'1deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
  

In [77]:
## Data preporcessing and transforming into images
for file_pd in files_pd:

    # Data preporcessing
    # Interpolate the NaN values with linear
    data_pd  = pd.read_excel(file_pd+'/2_dps.xlsx').interpolate()
    t = data_pd[data_pd['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
    data_eye_pd_first = data_pd[t[0]+902:t[0]+14404] # first trial index
    data_eye_pd_second = data_pd[t[2]+902:t[2]+14404] # second trail index

    # First Trial
    outlier_r = outlier_remove(data_eye_pd_first,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_pd_first,'GazePointLeftX (ADCSpx)')

    
    ## Transform times series to images
    # Compute Gramian angular fields, GASF
    # #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_pd_first.drop(data_eye_pd_first.index[data_eye_pd_first['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_first['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'2deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_pd_first.drop(data_eye_pd_first.index[data_eye_pd_first['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_first['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'2deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'2deg'+'right.png',s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'2deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'2deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Second Trial
    data_eye_pd_second_right = data_eye_pd_second['GazePointRightX (ADCSpx)']
    data_eye_pd_second_left = data_eye_pd_second['GazePointLeftX (ADCSpx)']

    # Compute Gramian angular fields, GASF
    #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_pd_second.drop(data_eye_pd_second.index[data_eye_pd_second['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_second['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'2deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_pd_second.drop(data_eye_pd_second.index[data_eye_pd_second['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_second['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'2deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #Combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'2deg'+'right.png',s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'2deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'2deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
  

In [ ]:
## Data preporcessing and transforming into images
for file_pd in files_pd:

    # Data preporcessing
    # Interpolate the NaN values with linear
    data_pd  = pd.read_excel(file_pd+'/4_dps.xlsx').interpolate()
    t = data_pd[data_pd['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
    data_eye_pd_first = data_pd[t[0]+902:t[0]+7804] # first trial index
    data_eye_pd_second = data_pd[t[2]+902:t[2]+7804] # second trail index

    # First Trial
    outlier_r = outlier_remove(data_eye_pd_first,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_pd_first,'GazePointLeftX (ADCSpx)')

    
    ## Transform times series to images
    # Compute Gramian angular fields, GASF
    # #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_pd_first.drop(data_eye_pd_first.index[data_eye_pd_first['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_first['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'4deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_pd_first.drop(data_eye_pd_first.index[data_eye_pd_first['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_first['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'4deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'4deg'+'right.png',s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'4deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'4deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Second Trial
    data_eye_pd_second_right = data_eye_pd_second['GazePointRightX (ADCSpx)']
    data_eye_pd_second_left = data_eye_pd_second['GazePointLeftX (ADCSpx)']

    # Compute Gramian angular fields, GASF
    #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_pd_second.drop(data_eye_pd_second.index[data_eye_pd_second['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_second['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'4deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_pd_second.drop(data_eye_pd_second.index[data_eye_pd_second['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_second['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'4deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #Combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'4deg'+'right.png',s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'4deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'4deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
  

In [43]:
## Data preporcessing and transforming into images
for file_pd in files_pd:

    # Data preporcessing
    # Interpolate the NaN values with linear
    data_pd  = pd.read_excel(file_pd+'/6_dps.xlsx').interpolate()
    t = data_pd[data_pd['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
    data_eye_pd_first = data_pd[t[0]+902:t[0]+5404] # first trial index
    data_eye_pd_second = data_pd[t[2]+902:t[2]+5404] # second trail index

    # First Trial
    outlier_r = outlier_remove(data_eye_pd_first,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_pd_first,'GazePointLeftX (ADCSpx)')

    
    ## Transform times series to images
    # Compute Gramian angular fields, GASF
    # #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_pd_first.drop(data_eye_pd_first.index[data_eye_pd_first['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_first['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_pd_first.drop(data_eye_pd_first.index[data_eye_pd_first['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_first['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'right.png',s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Second Trial
    data_eye_pd_second_right = data_eye_pd_second['GazePointRightX (ADCSpx)']
    data_eye_pd_second_left = data_eye_pd_second['GazePointLeftX (ADCSpx)']

    # Compute Gramian angular fields, GASF
    #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_pd_second.drop(data_eye_pd_second.index[data_eye_pd_second['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_second['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_pd_second.drop(data_eye_pd_second.index[data_eye_pd_second['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_second['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #Combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'right.png',s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
  

In [ ]:
## Data preporcessing and transforming into images
for file_pd in files_pd:

    # Data preporcessing
    # Interpolate the NaN values with linear
    data_pd  = pd.read_excel(file_pd+'/8_dps.xlsx').interpolate()
    t = data_pd[data_pd['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
    data_eye_pd_first = data_pd[t[0]+902:t[0]+4204] # first trial index
    data_eye_pd_second = data_pd[t[2]+902:t[2]+4204] # second trail index

    # First Trial
    outlier_r = outlier_remove(data_eye_pd_first,'GazePointRightX (ADCSpx)')
    outlier_l = outlier_remove(data_eye_pd_first,'GazePointLeftX (ADCSpx)')

    
    ## Transform times series to images
    # Compute Gramian angular fields, GASF
    # #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_pd_first.drop(data_eye_pd_first.index[data_eye_pd_first['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_first['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'8deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_pd_first.drop(data_eye_pd_first.index[data_eye_pd_first['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_first['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'8deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'8deg'+'right.png',s_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'8deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_pd))+'_'+'1'+'8deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Second Trial
    data_eye_pd_second_right = data_eye_pd_second['GazePointRightX (ADCSpx)']
    data_eye_pd_second_left = data_eye_pd_second['GazePointLeftX (ADCSpx)']

    # Compute Gramian angular fields, GASF
    #right_x
    fig = plt.figure()
    for i in outlier_r: data_eye_pd_second.drop(data_eye_pd_second.index[data_eye_pd_second['GazePointRightX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_second['GazePointRightX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'8deg'+'right.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #left_x
    fig = plt.figure()
    for i in outlier_l: data_eye_pd_second.drop(data_eye_pd_second.index[data_eye_pd_second['GazePointLeftX (ADCSpx)'] == i],inplace=True)
    X = data_eye_pd_second['GazePointLeftX (ADCSpx)'].interpolate()
    X = X.dropna()
    X = np.array([X])
    gasf = GramianAngularField(image_size=24,method='summation')
    X_gasf = gasf.fit_transform(X)
    plt.axis('off')
    plt.imshow(X_gasf[0], cmap ="rainbow")
    plt.savefig(s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'8deg'+'left.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    #Combine two images
    fig = plt.figure()
    images = [s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'8deg'+'right.png',s_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'8deg'+'left.png']
    output = concat_n_images(images)
    plt.axis('off')
    plt.imshow(output)
    plt.savefig(c_path+'/'+str(os.path.basename(file_pd))+'_'+'2'+'8deg'+'combine.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
  